# Introducing XGA Products

In this tutorial I will briefly touch on XGA's product classes, which are a key part of the internal makeup of the module. They act as an interface between XGA's functions and the data in different types of X-ray data (images, exposure maps, spectra etc.), and contain many useful and convenient methods for analysis and the extraction of information.

I will not be demonstrating specific abilities of different product types in this tutorial, there are too many to go into here, but I will give an overview of the purpose and important abilities of all the product classes built into XGA.

In [1]:
from astropy.units import Quantity

from xga.sources import GalaxyCluster

## What are products?

Most XGA products are what we use to wrap various types of data produced by XMM's SAS routines, as well as providing various product specific functionality. Generally, the user is unlikely to ever define a product instance themselves, internal methods in a source object and the SAS wrapper functions will define, check, and store the products. Those products that are generated by XGA's SAS interface have an added ability to parse the stderr output of the SAS routines used to generate them, then flag any recognised errors (by comparing to an archive of known SAS errors).

There are some XGA products that are not wrappers for SAS generated data products, but are purely for storing and providing access to XGA generated data. These are not based on the same superclass as the other products, but are stored in the same way.

Products instantiated by XGA are immediately 'associated' with a source object, though beyond storing the name of the source object in the product's internal structure, they have no knowledge or awareness of the source and its properties. This was by design, to make sure that any functionality built into a product would work regardless of the type of source, and even if it was defined independently of any source at all.

## What types of product are there?

* **BaseProduct** - The superclass for many of the standard XGA product classes, there isn't really any reason for a BaseProduct to be declared.

    * **EventList** - A very simple product class which differs only slightly from the BaseProduct class, its only used to store path and header information for the XMM event lists that the configuration file points XGA at.

    * **Image** - An extremely useful product with many extra features, it is used to wrap fits images. The data and header information are read into memory (when required), and can be accessed with properties and attributes of an Image instance. A view method (to look at the image and overlay extra information), and a coordinate transformation method are examples of the built-in functionality.

        * **ExpMap** - A subclass of the XGA Image class, this is a very simple extension to Image that adds a method to easily retrieve an exposure time at a given angular or pixel coordinate.

        * **RateMap** - The class that most photometric analyses are based around, also a subclass of Image. Instantiating this class requires the user pass matching Image and ExpMap instances (same ObsID, instrument etc.) A count rate map is then calculated from this information. This has several added methods, including the ability to retrieve a count rate at a given coordinate, and different peak finding methods.

        * **PSF** - This wraps two-dimensional PSF images generated by a routine such as [psfgen](https://xmm-tools.cosmos.esa.int/external/sas/current/doc/psfgen/index.html), and can be used to PSF correct images and ratemaps. Few methods have been added, and it is unlikely a user will ever need to interact directly with this. The added functionality here is the ability to resample the PSF at a scale provided by a passed in Image object.
    
    * **Spectrum** - A complicated sub-class of BaseProduct, it wraps and stores a spectrum generated by a SAS routine, including storing paths to all of the other files necessary to analyse it (e.g. RMF, ARF, background spectrum). When a source that has had spectra associated with it is passed to an XGA XSPEC function, the fit results are added to the spectrum objects. This has methods to retrieve fit results, as well as view the fitted Spectrum.
    
    
    
* **BaseAggregateProduct** - Another base product class, this one is designed to store a group of related XGA products.
    * **PSFGrid** - An XGA object used during the PSF correction of images and ratemaps, it stores a grid of PSF objects generated at different spatial positions on the XMM detectors. It provides access to those PSFs on request from the PSF correction function, and context as to which PSF was generated at which position.
    
    * **AnnularSpectra** - This holds sets of XGA Spectrum products generated in concentric annuli, *but is not yet fully implemented*.



* **BaseProfile1D** - Here we move to the special products that don't wrap existing X-ray data products, these are entirely generated by XGA. This class is designed to store, fit, and generate plots of different 1D profiles. The user should never declare an instance of this class, only the specific subclass that they need for their analysis.
    * **SurfaceBrightness1D** - Mostly meant for Galaxy Clusters, this class will store a 1D surface brightness profile, and enable the fitting of valid models such as a beta profile, or double beta profile.
    
    * **GasDensity1D** - This is meant to store a gas density profile as calculated by XGA, and includes methods to calculate a total gas mass within a given radius, as well as to generate a gas mass profile.
    
    * **GasMass1D** - A class for gas mass 1D profiles, which currently has no extra functionality over BaseProfile1D.


* **BaseAggregateProfile1D** - This is unlikely ever to actually have any specific subclasses, as awareness of the type of profile is not really necessary for its only job, which is to display multiple profiles on one axis. These can be generated just by adding multiple profile products together with the standard Python + operator


* **ScalingRelation** - The scaling relation products are unique in XGA, in that they are the only products that cannot be stored within a source, as they concern multiple sources (or no sources at all if declared from the literature), it would not make sense. These are produced by the scaling relation generation functions built into XGA, and in that case would contain both data and a fitted model. There are also several scaling relations from literature defined in XGA, these only contain the fitted model. A view method capable of producing publication quality plots is provided, along with many other convenient methods.


* **AggregateScalingRelation** - Just as the BaseAggregateProfile1D class was created purely to view multiple profiles on the same axes, this class is designed to enable scaling relations to be easily combined and viewed together. This class is instantiated by adding multiple ScalingRelation objects together.


## Retrieving products from a source

Here I will demonstrate how XGA stores product objects within a source, and how you can retrieve them if you wish to interact with the products directly. I will be demonstrate how to associate a product with a source object, as this is an internal mechanism with various checks and validation steps that the user should never have to interact with.

I'll define a GalaxyCluster source object corresponding to my favourite cluster, Abell 907 (**please note that the overdensity radii I've used here are approximate and should not be used for a scientific analysis**). We can see that this cluster is quite well observed, with three ObsIDs associated, with all instruments valid for each observation.

In [2]:
src = GalaxyCluster(149.59209, -11.05972, 0.16, r500=Quantity(1200, 'kpc'), r200=Quantity(1700, 'kpc'), 
                    name="A907")
src.info()


-----------------------------------------------------
Source Name - A907
User Coordinates - (149.59209, -11.05972) degrees
X-ray Peak - (149.59251340970866, -11.063958320861634) degrees
nH - 0.0534 1e+22 / cm2
Redshift - 0.16
XMM ObsIDs - 3
PN Observations - 3
MOS1 Observations - 3
MOS2 Observations - 3
On-Axis - 3
With regions - 3
Total regions - 69
Obs with one match - 3
Obs with >1 matches - 0
Images associated - 18
Exposure maps associated - 18
Combined Ratemaps associated - 1
Spectra associated - 0
R200 - 1700.0 kpc
R200 SNR - 1.44
R500 - 1200.0 kpc
R500 SNR - 2.68
-----------------------------------------------------



Products associatied with a source are stored in a protected attribute of the source, src.\_products. This essentially consists of a 

In [3]:
src._products

{'0404910601': {'pn': {'events': <xga.products.misc.EventList at 0x7f55b27202b0>,
   'bound_0.5-2.0': {'image': <xga.products.phot.Image at 0x7f55c8274940>,
    'expmap': <xga.products.phot.ExpMap at 0x7f55b26db070>,
    'ratemap': <xga.products.phot.RateMap at 0x7f55b26db160>},
   'bound_2.0-10.0': {'image': <xga.products.phot.Image at 0x7f55b26db1f0>,
    'expmap': <xga.products.phot.ExpMap at 0x7f55b26db190>,
    'ratemap': <xga.products.phot.RateMap at 0x7f55b26db0a0>}},
  'mos1': {'events': <xga.products.misc.EventList at 0x7f55e08fcc70>,
   'bound_0.5-2.0': {'image': <xga.products.phot.Image at 0x7f55b26db310>,
    'expmap': <xga.products.phot.ExpMap at 0x7f55b26db2e0>,
    'ratemap': <xga.products.phot.RateMap at 0x7f55b26db130>},
   'bound_2.0-10.0': {'image': <xga.products.phot.Image at 0x7f55b26db400>,
    'expmap': <xga.products.phot.ExpMap at 0x7f55b26db3d0>,
    'ratemap': <xga.products.phot.RateMap at 0x7f55b26db2b0>}},
  'mos2': {'events': <xga.products.misc.EventList at